# Embedding models
- https://python.langchain.com/docs/integrations/text_embedding/

## 比較各家 embedding模型

In [ ]:
!pip install -qU langchain-nvidia-ai-endpoints

In [6]:
# embedding模型設定
# https://build.nvidia.com/nvidia/nv-embed-v1
# nvapi-xxx
import getpass
import os
if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

from langchain.chat_models import init_chat_model
llm = init_chat_model("meta/llama-4-maverick-17b-128e-instruct", model_provider="nvidia")
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
embeddings = NVIDIAEmbeddings(model="nvidia/nv-embed-v1")

Enter API key for NVIDIA:  ········


D:\uv\langchaing\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:212: UserWarning: Found meta/llama-4-maverick-17b-128e-instruct in available_models, but type is unknown and inference may fail.
  warnings.warn(


In [ ]:
embeddings.embed_query("Hello, world!")

In [ ]:
!pip install -qU langchain-voyageai

In [ ]:
# embedding模型設定
# https://www.voyageai.com/
# https://dashboard.voyageai.com/api-keys
# pa-
import getpass
import os

if not os.environ.get("VOYAGE_API_KEY"):
  os.environ["VOYAGE_API_KEY"] = getpass.getpass("Enter API key for Voyage AI: ")
    
from langchain_voyageai import VoyageAIEmbeddings
embeddings = VoyageAIEmbeddings(model="voyage-3")

In [ ]:
embeddings.embed_query("Hello, world!")

In [ ]:
!pip install -qU langchain-community

In [ ]:
# embedding模型設定
# https://jina.ai/
# jina_xxx
import getpass
import os
if not os.environ.get("JINA_API_KEY"):
  os.environ["JINA_API_KEY"] = getpass.getpass("Enter API key for Voyage AI: ")

from langchain_community.embeddings import JinaEmbeddings
embeddings = JinaEmbeddings(
    jina_api_key=os.environ["JINA_API_KEY"], model_name="jina-embeddings-v3"
)

In [ ]:
embeddings.embed_query("Hello, world!")

In [ ]:
## 範例

In [1]:
# embedding模型設定
# https://jina.ai/
# jina_xxx
import getpass
import os
if not os.environ.get("JINA_API_KEY"):
  os.environ["JINA_API_KEY"] = getpass.getpass("Enter API key for Voyage AI: ")

from langchain_community.embeddings import JinaEmbeddings
embeddings = JinaEmbeddings(
    jina_api_key=os.environ["JINA_API_KEY"], model_name="jina-embeddings-v3"
)

Enter API key for Voyage AI:  ········


In [2]:
documents = [
    "Caching embeddings enables the storage or temporary caching of embeddings, eliminating the necessity to recompute them each time.",
    "An LLMChain is a chain that composes basic LLM functionality. It consists of a PromptTemplate and a language model (either an LLM or chat model). It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.",
    "A Runnable represents a generic unit of work that can be invoked, batched, streamed, and/or transformed.",
]
documents_embds = embeddings.embed_documents(documents)
documents_embds[0][:5]

[0.08977351, -0.079159915, 0.009447592, 0.012913363, 0.059213284]

In [3]:
query = "What's an LLMChain?"
query_embd = embeddings.embed_query(query)
query_embd[:5]

[0.11538703, -0.03468972, 0.030259516, -0.010401461, 0.07359464]

## KNNRetriever
- https://python.langchain.com/docs/integrations/retrievers
- https://python.langchain.com/docs/integrations/retrievers/knn/

In [4]:
from langchain_community.retrievers import KNNRetriever

documents = [
    "Caching embeddings enables the storage or temporary caching of embeddings, eliminating the necessity to recompute them each time.",
    "An LLMChain is a chain that composes basic LLM functionality. It consists of a PromptTemplate and a language model (either an LLM or chat model). It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.",
    "A Runnable represents a generic unit of work that can be invoked, batched, streamed, and/or transformed.",
]

query = "What's an LLMChain?"

retriever = KNNRetriever.from_texts(documents, embeddings,k=2)

# retrieve the most relevant documents
result = retriever.invoke(query)
top1_retrieved_doc = result[0].page_content  # return the top1 retrieved result

print(top1_retrieved_doc)

An LLMChain is a chain that composes basic LLM functionality. It consists of a PromptTemplate and a language model (either an LLM or chat model). It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.


In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


#retriever = KNNRetriever.from_texts(documents, embeddings,k=2)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)
query = "What's an LLMChain?"
chain.invoke({"input": query})

{'input': "What's an LLMChain?",
 'context': [Document(metadata={}, page_content='An LLMChain is a chain that composes basic LLM functionality. It consists of a PromptTemplate and a language model (either an LLM or chat model). It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.'),
  Document(metadata={}, page_content='A Runnable represents a generic unit of work that can be invoked, batched, streamed, and/or transformed.')],
 'answer': 'An LLMChain is a chain that composes basic LLM functionality, consisting of a PromptTemplate and a language model. It formats the prompt using input and memory key values, and returns the LLM output. It combines these elements to facilitate language model interactions.'}

# BM25

In [9]:
!uv pip install -qU  rank_bm25

In [10]:
from langchain_community.retrievers import BM25Retriever
retriever = BM25Retriever.from_texts(documents,k=2)
retriever.invoke(query)

[Document(metadata={}, page_content='An LLMChain is a chain that composes basic LLM functionality. It consists of a PromptTemplate and a language model (either an LLM or chat model). It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.'),
 Document(metadata={}, page_content='A Runnable represents a generic unit of work that can be invoked, batched, streamed, and/or transformed.')]

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


#retriever = KNNRetriever.from_texts(documents, embeddings,k=2)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)
query = "What's an LLMChain?"
chain.invoke({"input": query})

{'input': "What's an LLMChain?",
 'context': [Document(metadata={}, page_content='An LLMChain is a chain that composes basic LLM functionality. It consists of a PromptTemplate and a language model (either an LLM or chat model). It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.'),
  Document(metadata={}, page_content='A Runnable represents a generic unit of work that can be invoked, batched, streamed, and/or transformed.')],
 'answer': 'An LLMChain is a chain that composes basic LLM functionality, consisting of a PromptTemplate and a language model. It formats the prompt template using input key values and memory key values, then passes the formatted string to the LLM. It returns the LLM output.'}

##   🧠 作業三：語意理解任務中的 Embedding 實驗設計

**目標：**

* 分析 embedding 模型在不同語境下的表現
* 比較語意相近但字面不同的例子
* 比較不同 embedding 模型在相同任務上的表現 (可選擇)

**內容：**

1.  **設計句子對：**
    * 設計十組句子對，每組包含以下至少兩種語意關係的組合：
        * **同義：** 「他很開心」 vs 「他感到愉快」
        * **近義：** 「這輛車很新」 vs 「這輛車是全新的」
        * **反義：** 「他很高」 vs 「他很矮」
        * **上位/下位詞：** 「狗是一種動物」 vs 「黃金獵犬是一種狗」
        * **因果關係：** 「因為下雨，所以路很濕」 vs 「路很濕是因為下雨」
        * (鼓勵加入複雜句或多義詞)
    * 確保每組句子對中，至少有一對是語意相近，另一對是語意相異。

2.  **計算相似度：**
    * 選擇一個或多個 embedding 模型 (例如： notebook 中提到的或其他你感興趣的模型)。
    * 使用選擇的 embedding 模型計算每組句子對之間的相似度。
    * 建議使用餘弦相似度 (Cosine Similarity) 作為計算方法。

3.  **分析與評估：**
    * 分析模型的語意理解能力：
        * 模型是否能有效分辨語意相近與語意相異的句子對？
        * 模型對於不同語意關係的處理能力是否有差異？
        * 模型在處理複雜句或多義詞時的表現如何？
    * 使用以下指標評估模型表現：
        * **準確度：** 模型是否能正確區分語意相近和語意相異的句子對？ (可自行定義判斷標準)
        * **排序能力：** 如果將所有句子對依相似度排序，語意更相近的句子對是否排在更前面？
    * (如果選擇多個 embedding 模型) 比較不同模型之間的差異。

**評分標準：**

* 句子對設計的完整性與多樣性 (40%)
* 相似度計算的正確性 (30%)
* 分析與評估的深度 (30%)
* 報告的清晰度與完整性 (20%)

